![Coleridge Initiative Logo](images/CI_horizontal.png)

<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>

Contributors: Ghani, Rayid, Frauke Kreuter, Julia Lane, Brian Kim, Adrianne Bradford, Alex Engler, Nicolas Guetta Jeanrenaud, Graham Henke, Daniela Hochfellner, Clayton Hunter, Avishek Kumar, Jonathan Morgan, Ekaterina Levitskaya.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python-Setup" data-toc-modified-id="Python-Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python Setup</a></span></li><li><span><a href="#Data-Import" data-toc-modified-id="Data-Import-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Import</a></span></li><li><span><a href="#Record-Linkage" data-toc-modified-id="Record-Linkage-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Record Linkage</a></span><ul class="toc-item"><li><span><a href="#Indexing" data-toc-modified-id="Indexing-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Indexing</a></span></li></ul></li><li><span><a href="#Record-Comparison" data-toc-modified-id="Record-Comparison-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Record Comparison</a></span><ul class="toc-item"><li><span><a href="#Classification" data-toc-modified-id="Classification-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Classification</a></span></li></ul></li><li><span><a href="#References-and-Further-Readings" data-toc-modified-id="References-and-Further-Readings-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>References and Further Readings</a></span><ul class="toc-item"><li><span><a href="#Record-Linkage" data-toc-modified-id="Record-Linkage-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Record Linkage</a></span></li><li><span><a href="#Record-Linkage-Python-package" data-toc-modified-id="Record-Linkage-Python-package-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Record Linkage Python package</a></span></li><li><span><a href="#String-Comparators" data-toc-modified-id="String-Comparators-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>String Comparators</a></span></li><li><span><a href="#Fellegi-Sunter-Record-Linkage" data-toc-modified-id="Fellegi-Sunter-Record-Linkage-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Fellegi-Sunter Record Linkage</a></span></li></ul></li></ul></div>

## Python Setup

In [ ]:
# data import and manipulation
import pandas as pd

# record linkage and preprocessing
import recordlinkage as rl

## Data Import

Read-in previously cleaned datasets.

In [ ]:
# Read-in files with cleaned inventors (PatentsView) and principal investigators (FedRePORTER) names
inventors = pd.read_csv('../Data/inventors_cleaned.csv')
pi = pd.read_csv('../Data/pi_cleaned.csv')

In [ ]:
# View first rows of the table
inventors.head()

In [ ]:
# View first rows of the table
pi.head()

In [ ]:
# Read-in files with cleaned organization names in PatentsView (assignees) and FedRePORTER (organizations)
assignees = pd.read_csv('../Data/assignees_cleaned.csv')
organizations = pd.read_csv('../Data/organizations_cleaned.csv')

In [ ]:
# View first rows of the table
assignees.head()

In [ ]:
# View first rows of the table
organizations.head()

## Record Linkage

The `recordlinkage` package is a quite powerful tool for you to use when you want to link records within a dataset or across multiple datasets. We've already done some pre-processing and then tried deterministic matching.

However, as we have seen in the previous notebook, we might want to consider how strict we want our matching to be. For example, we want to make sure that we catch any typos or common misspellings, but we want to avoid relaxing the matching condition to the point that anything will match anything. 

### Indexing

Indexing allows us to create candidate links, which basically means identifying pairs of data rows which might refer to the same real world entity. This is also called the comparison space (matrix). There are different ways to index data. The easiest is to create a full index and consider every pair a match. This is also the least efficient method, because we will be comparing every row of one dataset with every row of the other dataset.

If we had 10,000 records in data frame A and 100,000 records in data frame B, we would have 1,000,000,000 candidate links. You can see that comparing over a full index is getting inefficient when working with big data.

We can do better if we actually include our knowledge about the data to eliminate bad link from the start. This can be done through blocking. The `recordlinkage` package gives you multiple options for this. For example, you can block by using variables, which means only links exactly equal on specified values will be kept. 

Here we will start by blocking on `city` and `state`, to narrow down the number of candidate links.

You can try and see how the number of candidate links change when blocking on more or less variables.

In [ ]:
indexerBL = rl.BlockIndex(on=['city', 'state'])
candidate_links = indexerBL.index(inventors, pi)

In [ ]:
len(candidate_links)

In [ ]:
candidate_links[:10]

Let's check the first pair of candidate links blocked on city and state: (0, 85)

In [ ]:
inventors.iloc[0]

In [ ]:
pi.iloc[85]

In [ ]:
# Now, in addition to blocking on city and state, we can also try blocking on first name
indexerBL = rl.BlockIndex(on=['name_first','city', 'state'])
candidate_links = indexerBL.index(inventors, pi)

In [ ]:
len(candidate_links)

## Record Comparison

After you have created a set of candidate links, you’re ready to begin comparing the records associated with each candidate link. In `recordlinkage` package you must initiate a Compare object prior to performing any comparison functionality between records. This object stores both dataframes, the candidate links, and a vector containing comparison results. Further, the Compare object contains the methods for performing comparisons. The code block below initializes the comparison object.

In [ ]:
# Initiate compare object 
compare_cl = rl.Compare()

Currently there are five specific comparison methods within recordlinkage: `Compare.exact()`, `Compare.string()`, `Compare.numeric()`, `Compare.geo()`, and `Compare.date()`. 

The `Compare.exact()` method is simple: if two values are an exact match a comparison score of 1 is returned, otherwise 0 is retured. 

The `Compare.string()` method is a bit more complicated and generates a score based on well-known string-comparison algorithms (for this example, Levenshtein or Jaro Winkler).

The Python `recordlinkage` toolkit uses the `jellyfish` package for the Jaro, Jaro-Winkler, Levenshtein and Damerau-Levenshtein algorithms: https://jellyfish.readthedocs.io/en/latest/comparison.html.

There can be a large difference in the performance of different string comparison algorithms. The Jaro and Jaro-Winkler methods are faster than the Levenshtein distance and much faster than the Damerau-Levenshtein distance.

String similarity measures and phonetic encodings are computationally expensive. After phonetic encoding of the string variables, exact comparing can be used instead of computing the string similarity of all record pairs. If the number of candidate pairs is much larger than the number of records in both datasets together, then consider using phonetic encoding of string variables instead of string comparison.

> Choose and compare only informative variables: not all variables may be worth comparing in a record linkage. Some variables do not discriminate the links of the non-links or do have only minor effects. These variables can be excluded. Only informative variables should be included.

For this example, Jaro-Winkler distance is used (specifically developed with record linkage applications in mind, faster to compute) - words with more characters in common have a higher Jaro-Winkler value than those with fewer characters in common. The Jaro–Winkler distance gives more favorable ratings to strings that match from the beginning. The output value is normalized to fall between 0 (complete dissimilar strirngs) and 1 (exact match on strings).

As you remember, we already did an exact matching on `city` and `state`, when we did the blocking above and created the candidate links.

We will use the string method to compare the organization names and their phonetic transcriptions.

We need to specify the respective columns with organization names in both datasets, the method, and the threshold. In this case, for all strings that have more than 85% in similarity, according to the Jaro-Winkler distance, a 1 will be returned, and otherwise 0.

In [ ]:
# Initiate compare object 
compare_cl = rl.Compare()

In [ ]:
compare_cl.string('name_first', 'name_first', method='jarowinkler', threshold=0.85, label='name_first')
compare_cl.string('name_last', 'name_last', method='jarowinkler', threshold=0.85, label='name_last')

#compare_cl.exact('state', 'state', label='state')

The comparing of record pairs starts when the `compute` method is called.

In [ ]:
indexerBL = rl.BlockIndex(on=['city', 'state'])
candidate_links = indexerBL.index(inventors, pi)

In [ ]:
len(candidate_links)

In [ ]:
## All attribute comparisons are stored in a DataFrame with horizontally the features and vertically the record pairs.
features = compare_cl.compute(candidate_links, inventors, pi)

In [ ]:
features.head(7)

In [ ]:
features.tail()

### Classification

Let's check how many records we get where one or more comparison attributes match.

In [ ]:
## Simple Classification: Check for how many attributes records are identical by summing the comparison results.
features.sum(axis=1).value_counts().sort_index(ascending=False)

We can make a decision now, and consider matches all those records which matched on all attributes in our case.

In [ ]:
matches = features[features.sum(axis=1) == 2]
print(len(matches))

Remember that for these matches we had an exact match on `city` and `state`, and more than 80% in similarity based on organization `first name` and `last name`

In [ ]:
matches.head()

Now let's merge these matches back to original dataframes.

Our `matches` dataframe has MultiIndex - two indices to the left which correspond to the `inventor` table and `pi` table respectively.

We can access each matching pair individually, for example, the first one:

In [ ]:
matches.index[0]

We can also do the following: first, put all the indices for the `inventors` table.

In [ ]:
matches.index[0][0]

We will pull all corresponding rows from the `inventors` table.

In [ ]:
inventors_results = []  # Create an empty list

for match in matches.index:  # For every pair in matches (index)
    df = pd.DataFrame(inventors.loc[[match[0]]])  # Get the location in the original table, convert to dataframe
    inventors_results.append(df)

In [ ]:
inventors_results[0]

Now we concatenate the list of dataframes into one dataframe.

In [ ]:
inventors_concat = pd.concat(inventors_results)

In [ ]:
inventors_concat.head()

We do the same for the `pi` table.

In [ ]:
pi_results = []  # Create an empty list

for match in matches.index:  # For every pair in matches (index)
    df = pd.DataFrame(pi.loc[[match[1]]])  # Get the location in the original table, convert to dataframe
    pi_results.append(df)
    
pi_concat = pd.concat(pi_results)

In [ ]:
pi_concat.head()

Now we need to combine two tables on the index - notice that our tables right now have indices from the original tables. We can reset the index using `.reset_index()`.

In [ ]:
inventors_concat = inventors_concat.reset_index()
pi_concat = pi_concat.reset_index()

Now our tables have the same index on which we can combine two tables.

In [ ]:
inventors_concat.head()

In [ ]:
pi_concat.head()

In [ ]:
# Drop the old index column
inventors_concat = inventors_concat.drop(columns=['index'])
pi_concat = pi_concat.drop(columns=['index'])

In [ ]:
# Drop other not relevant columns
inventors_concat = inventors_concat.drop(columns=['inventor_country'])
pi_concat = pi_concat.drop(columns=[' ORGANIZATION_COUNTRY'])

Now we concatenate these two tables using `.concat()`.

In [ ]:
matched = pd.concat([inventors_concat, pi_concat], axis=1)  # Specify axis=1 to concatenate horizontally

In [ ]:
matched[14:20]

Now that we have merged our matches together, examine them. Remember that we matched our strings on 85% similarity, according to the Jaro-Winkler distance,  and we blocked on city and state. Try using a different threshold. You can also use a different string-matching algorithm (please see below in References).

<span style="color:red">**Checkpoint: Record Linkage Decisions**</span>

What are some decisinos we had to make as we went through the record linkage process above? What if we had made different choices instead?

Try doing the record linkage with a few different options and see how many matches you get as you vary the approach. For example, try different string-matching algorithms or thresholds.

In [ ]:
# your code here...





## References and Further Readings

### Record Linkage

Lane, Julia, Ian Foster, Rayid Ghani, Ron S. Jarmin, Frauke Kreuter (editors), Big Data and Social Science: A Practical Guide to Methods and Tools, Chapman and Hall/CRC Press, 2016. https://coleridge-initiative.github.io/big-data-and-social-science/chap-link.html

### Record Linkage Python package
* `recordlinkage` Python package: https://recordlinkage.readthedocs.io/en/latest/index.html
    - Comparing records: https://recordlinkage.readthedocs.io/en/latest/ref-compare.html
    - Classification:
        - https://recordlinkage.readthedocs.io/en/latest/ref-classifiers.html,
        - https://recordlinkage.readthedocs.io/en/latest/notebooks/classifiers.html

### String Comparators

* GitHub page of `jellyfish`: https://github.com/jamesturk/jellyfish
* Descriptions of distances in `jellyfish`: https://jellyfish.readthedocs.io/en/latest/comparison.html
* Different distances that measure the differences between strings:
    - Levenshtein distance: https://en.wikipedia.org/wiki/Levenshtein_distance
    - Damerau–Levenshtein distance: https://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance
    - Jaro–Winkler distance: https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance
    - Hamming distance: https://en.wikipedia.org/wiki/Hamming_distance
    - Match rating approach: https://en.wikipedia.org/wiki/Match_rating_approach

### Fellegi-Sunter Record Linkage 

* Introduction to Probabilistic Record Linkage: http://www.bristol.ac.uk/media-library/sites/cmm/migrated/documents/problinkage.pdf
* Paper Review: https://www.cs.umd.edu/class/spring2012/cmsc828L/Papers/HerzogEtWires10.pdf

